In [ ]:
import numpy as np 
import pandas as pd
import os
import torch
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader,Dataset, sampler, random_split
from torchvision import models
!pip install timm # kaggle doesnt have it installed by default
import timm
from timm.loss import LabelSmoothingCrossEntropy # This is better than normal nn.CrossEntropyLoss
import matplotlib.pyplot as plt
%matplotlib inline
import sys
from tqdm import tqdm
import time
import copy
from os import listdir
from os.path import isfile, join
from PIL import Image
import glob

In [ ]:
#Método para obtener todas las especies de pájaros a partir de la estructura de carpetas
def get_classes(data_dir):
    all_data = datasets.ImageFolder(data_dir)
    return all_data.classes

In [ ]:
#Método para entrenar el modelo
def train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes, num_epochs=5):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print("-"*10)
        
        '''for phase in ['train', 'val']: # En principio no hemos usado fase de validación
            if phase == 'train':
                model.train() # model to training mode
            else:
                model.eval() # model to evaluate
            
            running_loss = 0.0
            running_corrects = 0.0
            
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'): # no autograd makes validation go faster
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) # used for accuracy
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step() # step at end of epoch
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc =  running_corrects.double() / dataset_sizes[phase]
            
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict()) # keep the best validation accuracy model
        print()'''
        for phase in ['train']: # En principio no hemos usado fase de validación
            model.train() # model to training mode
            
            running_loss = 0.0
            running_corrects = 0.0
            
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(True): # no autograd makes validation go faster
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) # used for accuracy
                    loss = criterion(outputs, labels)
                    
                    loss.backward()
                    optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            scheduler.step() # step at end of epoch
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc =  running_corrects.double() / dataset_sizes[phase]
            
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            if epoch_acc > best_acc:
                best_acc = epoch_acc
        print()
    time_elapsed = time.time() - since # slight error
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print("Best Val Acc: {:.4f}".format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def pre_image(image_path,model):
    #img = Image.open(image_path)
    mean = [0.485, 0.456, 0.406] 
    std = [0.229, 0.224, 0.225]
    transform_norm = transforms.Compose([transforms.ToTensor(), 
    transforms.Resize((224,224)),transforms.Normalize(mean, std)])
   # get normalized image
    img_normalized = transform_norm(image_path).float()
    img_normalized = img_normalized.unsqueeze(0)
   # input = Variable(image_tensor)
    img_normalized = img_normalized.to(device)
   # print(img_normalized.shape)
    with torch.no_grad():
        model.eval()
        output =model(img_normalized)
     # print(output)
        index = output.data.cpu().numpy().argmax()
        class_name = classes[index]
        return class_name

In [ ]:

 #train
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter()]), p=0.25),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
    transforms.RandomErasing(p=0.2, value='random')
])
train_data = datasets.ImageFolder('../input/iais22-birds/birds/birds', transform = transform_train)
train_loader = DataLoader(train_data, batch_size=256, shuffle=True, num_workers=4)
train_data_len = len(train_data)

In [ ]:
dataloaders = {
    "train": train_loader,
    #"val": val_loader
}
dataset_sizes = {
    "train": train_data_len,
    #"val": valid_data_len
}

In [ ]:
classes = get_classes("../input/iais22-birds/birds/birds")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
torch.backends.cudnn.benchmark = True
model = models.efficientnet_b1(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
n_inputs = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Linear(n_inputs,2048),
    nn.SiLU(),
    nn.Dropout(0.2),
    nn.Linear(2048, len(classes))
)

model = model.to(device)
print(model.classifier)

In [ ]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.11)
criterion = criterion.to(device)
optimizer = optim.AdamW(model.classifier.parameters(), lr=0.001)

In [ ]:
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.97)

In [ ]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, dataloaders, dataset_sizes)

In [ ]:

image_list = []
preds_id = []
for filename in glob.glob("../input/iais22-birds/submission_test/submission_test/*.jpg"): 
    im=Image.open(filename)
    id = os.path.basename(filename).split(".")[0]
    image_list.append(im)
    preds_id.append(id)

index=[]
preds = []
for f in image_list:
    i = image_list.index(f)+1
    predict_class = pre_image(f,model)
    index.append(i)
    preds.append(predict_class)
    if(i%500==0):
        print(i)

submission = pd.DataFrame(
    data =np.array([preds_id,preds ]).T, 
    columns = ["Id", "Category"]
)
submission.to_csv("submission.csv", index = False)
submission.head()